# Optimización de hiperparámetros

En nuestro modelo previo hemos conseguido un RMSE de 47328$ con un modelo RandomForestRegressor. Vamos a ajustar sus hiperparámetros para intentar reducir más ese error.

## Pasos previos

### Definición del *pipeline*

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
cat_pipeline = make_pipeline( # Pipeline for categorical features
    SimpleImputer(strategy="most_frequent"), # Impute missing values with the most frequent value
    OneHotEncoder(handle_unknown="ignore")) # One-hot encode the categorical features

In [3]:
class ClusterSimilarity(BaseEstimator, TransformerMixin): # Custom transformer to compute similarity with cluster center
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma # RBF kernel bandwidth
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10, 
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)

In [4]:
def column_ratio(X): # Custom transformer to compute the ratio of two columns
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in): # Custom function to name the output columns
    return ["ratio"]  # feature names out

def ratio_pipeline(): # Pipeline for ratio features (create new features by dividing two columns)
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())

In [5]:
preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]), # razón entre total_bedrooms y total_rooms (nueva feature)
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]), # razón entre total_rooms y households (nueva feature)
        ("people_per_house", ratio_pipeline(), ["population", "households"]), # razón entre population y households (nueva feature)
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]), # logaritmo de las columnas seleccionadas (para cambiar distribuciones sesgadas -skewed- por distribuciones normales)
        ("geo", cluster_simil, ["latitude", "longitude"]), # similitud con los clusters
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)), # pipeline categórico
    ],
    remainder=default_num_pipeline)  # one column remaining: housing_median_age

preprocessing

ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                             SimpleImputer(strategy='median')),
                                            ('standardscaler',
                                             StandardScaler())]),
                  transformers=[('bedrooms',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('functiontransformer',
                                                  FunctionTransformer(feature_names_out=<function ratio_name at 0x7bf398073880>,
                                                                      func=<function column_ratio...
                                 ['total_bedrooms', 'total_rooms', 'population',
                                  'households', 'median_income']),
                                ('geo', ClusterSimilarity(random_state=42),
                                 ['latitude', 'longitude']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7bf398076000>)])

In [6]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42)),
])

### Importación y preparación de datos

In [7]:
housing = pd.read_csv("./data/housing.csv")

In [8]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    
X_train = strat_train_set.drop("median_house_value", axis=1)
y_train = strat_train_set["median_house_value"].copy()

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

## Visualización de hiperparámetros

In [9]:
full_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7bf3980...
                                                 ('geo',
                                                  ClusterSimilarity(random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7bf398076000>)])),
                ('random_forest', RandomForestRegressor(random_state=42))])

Para ver los nombres de los hiperparámetros que se pueden ajustar, se puede usar el siguiente código:

In [10]:
for param in sorted(full_pipeline.get_params().keys()):
    print(param)

memory
preprocessing
preprocessing__bedrooms
preprocessing__bedrooms__functiontransformer
preprocessing__bedrooms__functiontransformer__accept_sparse
preprocessing__bedrooms__functiontransformer__check_inverse
preprocessing__bedrooms__functiontransformer__feature_names_out
preprocessing__bedrooms__functiontransformer__func
preprocessing__bedrooms__functiontransformer__inv_kw_args
preprocessing__bedrooms__functiontransformer__inverse_func
preprocessing__bedrooms__functiontransformer__kw_args
preprocessing__bedrooms__functiontransformer__validate
preprocessing__bedrooms__memory
preprocessing__bedrooms__simpleimputer
preprocessing__bedrooms__simpleimputer__add_indicator
preprocessing__bedrooms__simpleimputer__copy
preprocessing__bedrooms__simpleimputer__fill_value
preprocessing__bedrooms__simpleimputer__keep_empty_features
preprocessing__bedrooms__simpleimputer__missing_values
preprocessing__bedrooms__simpleimputer__strategy
preprocessing__bedrooms__standardscaler
preprocessing__bedrooms_

## *Grid Search*

Para evitar lo tedioso de ir modificando manualmente los hiperparámetros de un modelo hasta encontrar los que mejor resultado den, podemos definir todos los valores de hiperparámetros que queremos probar y programar que se prueben todas las combinaciones posibles.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'preprocessing__geo__n_clusters': [5, 8, 10], # number of clusters for the geo transformer
     'random_forest__max_features': [4, 6, 8]}, # number of features to consider when looking for the best split
    {'preprocessing__geo__n_clusters': [10, 15],
     'random_forest__max_features': [6, 8, 10]},
]
grid_search = GridSearchCV(
    estimator = full_pipeline,
    param_grid = param_grid, 
    cv=3,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
    )

_ = grid_search.fit(X_train, y_train)

/media/NOCText4/Alejandro/miniconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


El parámetro ```param_grid``` es una lista de diccionarios, cada uno de los cuales contiene los valores de los hiperparámetros que queremos probar. En este caso se están probando primero 3 valores para el número de clusters y 3 para el número de *features* consideradas en cada división. Después se están probando 2 valores para el número de clusters y 3 para el número de *features*. En total se están probando 3*3 + 2*3 = 15 combinaciones de hiperparámetros.

Por otra parte, el parámetro ```n_jobs``` permite paralelizar la búsqueda de hiperparámetros indicando el número de procesadores que se usarán; el valor -1 indica que se usarán todos los procesadores disponibles. Este mismo parámetros se puede usar en el modelo de RandomForestRegressor para paralelizar la construcción de los árboles, pero hay que tener cuidado si se hacen ambas cosas, ya que si se paraleliza cada búsqueda de hiperparámetros, que es cada una una ejecución de modelo, y ese modelo a su vez paraleliza la construcción de los árboles, se estarían multiplicando el total de ejecuciones. El total de n_jobs de RandomForestRegressor multiplicado por el número de búsquedas no puede superar el número de cores físicos del equipo. En general, es mejor paralelizar la búsqueda de hiperparámetros y no la construcción de los árboles; por eso, en este caso se ha optado por dejar RandomForestRegressor con el valor por defecto ```n_jobs=None```, que asigna 1 core por cada árbol y el máximo para GridSearchCV con el valor ```n_jobs=-1```.

Podemos ahora ver los mejores hiperparámetros encontrados.

In [12]:
grid_search.best_params_

{'preprocessing__geo__n_clusters': 15, 'random_forest__max_features': 6}

Vemos que el mejor modelo tiene 15 *clusters*. Dado que es el valor más alto que se ha probado, tendría sentido hacer nuevas pruebas con otros valores más grandes.

Tambien nos devuelve el mejor modelo encontrado:

In [13]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('bedrooms',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out=<function ratio_name at 0x7bf3980...
                                                  ClusterSimilarity(n_clusters=15,
                                                                    random_state=42),
                                                  ['latitude', 'longitude']),
                                                 ('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7bf3980d2ba0>)])),
                ('random_forest',
                 RandomForestRegressor(max_features=6, random_state=42))])

También podemos ver el resultado de cada combinación de hiperparámetros probada durante la búsqueda:

In [14]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

# Seleccionamos las columnas que queremos mostrar
cv_res = cv_res[["param_preprocessing__geo__n_clusters",
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]

# Cambiamos el nombre de las columnas para simplificar
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = ["n_clusters", "max_features"] + score_cols
# Limpiamos la medida de score
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)
cv_res.head()

,n_clusters,max_features,split0,split1,split2,mean_test_rmse
12,15,6,43400,44475,45021,44299
13,15,8,43662,44478,45495,44545
14,15,10,44231,45167,46182,45193
7,10,6,44437,45257,46452,45382
9,10,6,44437,45257,46452,45382


## Randomized Search

En lugar de probar todas las combinaciones posibles de hiperparámetros, RandomizedSearchCV permite probar un número determinado de combinaciones aleatorias. Esto aporta ciertas ventajas:

- Eficiencia computacional: La "maldición de la dimensionalidad" hace que Grid Search sea computacionalmente inviable muy rápidamente. Con más de 3 o 4 hiperparámetros con unas pocas opciones cada uno, el número total de combinaciones a probar explota. Randomized Search permite fijar un presupuesto computacional (número de iteraciones) independientemente del número de hiperparámetros, haciéndolo factible para problemas complejos

- Efectividad en dimensiones altas: para muchas funciones objetivo (como el rendimiento de un modelo), solo unos pocos hiperparámetros tienen un impacto significativo. Al muestrear aleatoriamente combinaciones, tiene una mayor probabilidad de probar valores diversos en las dimensiones importantes, mientras que GS gasta mucho esfuerzo probando sistemáticamente valores en dimensiones que apenas afectan el resultado.

- Manejo de parámetros continuos: RS maneja de forma natural los parámetros continuos muestreando de una distribución (ej., uniforme, log-uniforme). GS requiere discretizar el rango, lo que es artificial y puede fácilmente omitir el valor óptimo real si cae entre los puntos de la rejilla.

Es por esto que RandomizedSearchCV es en general más eficiente que GridSearchCV para problemas de alta dimensión, con muchos hiperparámetros o donde no tengamos una idea clara de los rangos de valores de los hiperparámetros.

In [20]:
list(range(3,50))

range(3, 50)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
    'preprocessing__geo__n_clusters': randint(low=3, high=50),
    'random_forest__max_features': randint(low=2, high=20)
    }

rnd_search = RandomizedSearchCV(
    full_pipeline,
    param_distributions=param_distribs,
    n_iter=10, # número de iteraciones
    cv=3,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1
    )

_ = rnd_search.fit(X_train, y_train)

```scipy.stats.randint()``` devuelve un objeto que contiene la distribución de probabilidad de la variable aleatoria discreta. RandomizedSearchCV lo usa para muestrear aleatoriamente valores de los hiperparámetros.

El parámetro ```n_iter``` es el número de iteraciones que se realizarán. En este caso se están probando 10 combinaciones de hiperparámetros.

In [16]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res = cv_res[["param_preprocessing__geo__n_clusters",
                 "param_random_forest__max_features", "split0_test_score",
                 "split1_test_score", "split2_test_score", "mean_test_score"]]
score_cols = ["split0", "split1", "split2", "mean_test_rmse"]
cv_res.columns = ["n_clusters", "max_features"] + score_cols
cv_res[score_cols] = -cv_res[score_cols].round().astype(np.int64)
cv_res.head()

,n_clusters,max_features,split0,split1,split2,mean_test_rmse
1,45,9,41482,42969,43230,42560
8,32,7,41890,43529,43621,43013
5,42,4,41955,44061,43505,43174
0,41,16,42614,43528,44232,43458
2,23,8,42880,43378,44483,43580


Hemos conseguido mejorar nuestro modelo bajando el RMSE hasta 42560$ definiendo 45 clusters y considerando 9 features para cada división.

## Evaluando el modelo final en el conjunto de test

In [17]:
from sklearn.metrics import root_mean_squared_error

final_predictions = rnd_search.best_estimator_.predict(X_test)

final_rmse = root_mean_squared_error(y_test, final_predictions)
print(final_rmse)

39565.359517373756
